## Genome Assembly

### High-Level Description (Conceptual)

**Goal**: Reconstruct the original DNA sequence from a list of overlapping fragments (k-mers).

**Approach**:

1. **De Bruijn Graph**:

   * Represent k-mers as edges.
   * Nodes are (k-1)-mers (prefixes/suffixes).
   * Find an **Eulerian path**: a path that visits every edge exactly once.
   * Rebuild the sequence by following this path.

2. **Overlap Graph**:

   * Represent k-mers as nodes.
   * Add an edge from node A to node B if the suffix of A matches the prefix of B (length k-1).
   * Find a **Hamiltonian path**: a path that visits every node exactly once.
   * Reconstruct the sequence by joining overlapping fragments along the path.

---

###  Low-Level Description (Implementation)

**De Bruijn Algorithm**:

* For each k-mer, add an edge from `prefix(k-mer)` to `suffix(k-mer)`.
* Ensure the graph is *nearly balanced* (1 start, 1 end node).
* Add a temporary edge from end to start.
* Use **Hierholzer’s algorithm** to find an Eulerian cycle.
* Remove the temporary edge to get the Eulerian path.
* Reconstruct the sequence by concatenating characters from each node.

**Overlap Graph Algorithm**:

* Label each fragment uniquely (e.g., `"ATG-1"`).
* For every pair of fragments, add an edge if `suffix(A) == prefix(B)`.
* Use **backtracking** to search for a Hamiltonian path.
* Rebuild the sequence using the first full fragment and last characters from the rest.


In [1]:
def composition(k, seq):
    return sorted([seq[i:i+k] for i in range(len(seq)-k+1)])

def prefix(seq): return seq[:-1]
def suffix(seq): return seq[1:]

In [2]:
class MyGraph:
    def __init__(self): 
        self.graph = {}
    def add_vertex(self, v): 
        self.graph.setdefault(v, [])
    def add_edge(self, o, d): 
        self.add_vertex(o); 
        self.add_vertex(d); 
        self.graph[o].append(d)
    def print_graph(self): 
        [print(v, '->', self.graph[v]) for v in self.graph]
    def get_nodes(self): 
        return list(self.graph)
    def in_degree(self, v): 
        return sum([lst.count(v) for lst in self.graph.values()])
    def out_degree(self, v): 
        return len(self.graph[v])

In [3]:
class DeBruijnGraph(MyGraph):
    def __init__(self, frags):
        super().__init__()
        # Add edges where each k-mer contributes an edge from its prefix to its suffix
        for seq in frags:
            self.add_edge(prefix(seq), suffix(seq))

    def check_nearly_balanced_graph(self):
        # Identify if the graph is nearly balanced:
        # One node with out-degree = in-degree + 1 (start)
        # One node with in-degree = out-degree + 1 (end)
        res = None, None  # (start, end)
        for n in self.graph:
            indeg = self.in_degree(n)
            outdeg = self.out_degree(n)
            if indeg - outdeg == 1:
                res = res[0], n  # candidate to be end node
            elif outdeg - indeg == 1:
                res = n, res[1]  # candidate to be start node
            elif indeg != outdeg:
                return None, None  # not balanced or nearly balanced
        return res

    def eulerian_path(self):
        # Find a Eulerian path using Hierholzer's algorithm
        start, end = self.check_nearly_balanced_graph()
        if not start or not end:
            return None

        # Add a temporary edge to make the graph Eulerian
        self.add_edge(end, start)

        path = []
        stack = [start]
        # Copy of graph edges to allow mutation during traversal
        edges = {u: list(vs) for u, vs in self.graph.items()}

        while stack:
            u = stack[-1]
            if edges.get(u):
                stack.append(edges[u].pop())
            else:
                path.append(stack.pop())

        path.reverse()

        # Remove the temporary edge to recover the original path
        for i in range(len(path) - 1):
            if path[i] == end and path[i + 1] == start:
                return path[i + 1:] + path[1:i + 1]

        return None

    def seq_from_path(self, path):
        # Reconstruct the original sequence from a Eulerian path
        if not path:
            return None
        return path[0] + ''.join(n[-1] for n in path[1:])


In [4]:
class OverlapGraph(MyGraph):
    def __init__(self, frags):
        super().__init__()

        # Add unique suffix to each fragment to handle duplicates
        self.frags = [f"{f}-{i}" for i, f in enumerate(frags, 1)]

        # Add all vertices to the graph
        for f in self.frags:
            self.add_vertex(f)

        # Add edges based on overlap: suffix of f1 matches prefix of f2
        for f1 in self.frags:
            s1 = suffix(f1.split('-')[0])  # suffix of the sequence
            for f2 in self.frags:
                if prefix(f2.split('-')[0]) == s1:
                    self.add_edge(f1, f2)

    def search_hamiltonian_path(self):
        # Try to find a Hamiltonian path using backtracking
        def bt(path):
            if len(path) == len(self.graph):
                return path
            for neighbor in self.graph[path[-1]]:
                if neighbor not in path:
                    res = bt(path + [neighbor])
                    if res:
                        return res
            return None

        # Attempt to start from every vertex
        for start in self.graph:
            res = bt([start])
            if res:
                return res
        return None

    def get_seq(self, node):
        # Extract the original sequence from node label (e.g., 'ATG-3' -> 'ATG')
        return node.split('-')[0]

    def seq_from_path(self, path):
        # Reconstruct sequence from Hamiltonian path
        if not path:
            return None
        return self.get_seq(path[0]) + ''.join(self.get_seq(n)[-1] for n in path[1:])


In [5]:
def run_all(seq, k):
    print("Original sequence:", seq)
    frags = composition(k, seq)
    print("k-mers:", frags)

    # De Bruijn Graph Method
    print("\n--- De Bruijn ---")
    dbg = DeBruijnGraph(frags)
    dbg.print_graph()
    path = dbg.eulerian_path()
    if path:
        print("Eulerian path found:", path)
        print("Reconstructed sequence:", dbg.seq_from_path(path))
    else:
        print("Eulerian path not found.")

    # Overlap Graph Method with repetitions
    print("\n--- Overlap with repetitions ---")
    og = OverlapGraph(frags)
    og.print_graph()
    path = og.search_hamiltonian_path()
    if path:
        print("Hamiltonian path found:", path)
        print("Reconstructed sequence:", og.seq_from_path(path))
    else:
        print("Hamiltonian path not found.")



In [6]:
run_all('ATGCAATGGTCTG', 3)

Original sequence: ATGCAATGGTCTG
k-mers: ['AAT', 'ATG', 'ATG', 'CAA', 'CTG', 'GCA', 'GGT', 'GTC', 'TCT', 'TGC', 'TGG']

--- De Bruijn ---
AA -> ['AT']
AT -> ['TG', 'TG']
TG -> ['GC', 'GG']
CA -> ['AA']
CT -> ['TG']
GC -> ['CA']
GG -> ['GT']
GT -> ['TC']
TC -> ['CT']
Eulerian path found: ['AT', 'TG', 'GG', 'GT', 'TC', 'CT', 'TG', 'GC', 'CA', 'AA', 'AT', 'TG']
Reconstructed sequence: ATGGTCTGCAATG

--- Overlap with repetitions ---
AAT-1 -> ['ATG-2', 'ATG-3']
ATG-2 -> ['TGC-10', 'TGG-11']
ATG-3 -> ['TGC-10', 'TGG-11']
CAA-4 -> ['AAT-1']
CTG-5 -> ['TGC-10', 'TGG-11']
GCA-6 -> ['CAA-4']
GGT-7 -> ['GTC-8']
GTC-8 -> ['TCT-9']
TCT-9 -> ['CTG-5']
TGC-10 -> ['GCA-6']
TGG-11 -> ['GGT-7']
Hamiltonian path found: ['ATG-2', 'TGC-10', 'GCA-6', 'CAA-4', 'AAT-1', 'ATG-3', 'TGG-11', 'GGT-7', 'GTC-8', 'TCT-9', 'CTG-5']
Reconstructed sequence: ATGCAATGGTCTG
